# Imports

In [47]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sns
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

#from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import OneHotEncoder

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.3.0
Sklearn  0.24.2


# Load Data

In [48]:
df = pd.read_csv('movies_utf8.csv')

In [49]:
df

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6815,0.0,Fox Searchlight Pictures,UK,Mandie Fletcher,Comedy,4750497.0,Absolutely Fabulous: The Movie,R,2016-07-22,91,5.4,Jennifer Saunders,9161,Jennifer Saunders,2016
6816,0.0,Siempre Viva Productions,USA,Paul Duddridge,Drama,28368.0,Mothers and Daughters,PG-13,2016-05-06,90,4.9,Selma Blair,1959,Paige Cameron,2016
6817,3500000.0,Warner Bros. Animation,USA,Sam Liu,Animation,3775000.0,Batman: The Killing Joke,R,2016-07-25,76,6.5,Kevin Conroy,36333,Brian Azzarello,2016
6818,0.0,Borderline Presents,USA,Nicolas Pesce,Drama,25981.0,The Eyes of My Mother,R,2016-12-02,76,6.2,Kika Magalhães,6947,Nicolas Pesce,2016


# Check Missing

In [50]:
df.isnull().sum()/len(df)

budget      0.0
company     0.0
country     0.0
director    0.0
genre       0.0
gross       0.0
name        0.0
rating      0.0
released    0.0
runtime     0.0
score       0.0
star        0.0
votes       0.0
writer      0.0
year        0.0
dtype: float64

# Data Type

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   budget    6820 non-null   float64
 1   company   6820 non-null   object 
 2   country   6820 non-null   object 
 3   director  6820 non-null   object 
 4   genre     6820 non-null   object 
 5   gross     6820 non-null   float64
 6   name      6820 non-null   object 
 7   rating    6820 non-null   object 
 8   released  6820 non-null   object 
 9   runtime   6820 non-null   int64  
 10  score     6820 non-null   float64
 11  star      6820 non-null   object 
 12  votes     6820 non-null   int64  
 13  writer    6820 non-null   object 
 14  year      6820 non-null   int64  
dtypes: float64(3), int64(3), object(9)
memory usage: 799.3+ KB


# Feature Extraction

## Total Rows 6820

## Strings Unique
- company 2179
- country 57
- director 2759
- genre 17
- rating 13
- released 2403
- star 2504
- writer 4199

# Tasks:
- DROP released [x]
- One Hot Encode all string data []

In [52]:
df.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [53]:
df.director.value_counts()

Woody Allen          33
Clint Eastwood       24
Steven Soderbergh    21
Steven Spielberg     21
Ron Howard           20
                     ..
Jun Falkenstein       1
Kamal Haasan          1
Jean-Loup Hubert      1
Roy Andersson         1
Nicole Garcia         1
Name: director, Length: 2759, dtype: int64

# Data Cleaning

# x & y

In [54]:
x = df.drop(columns=(['score', 'released', 'name']))
y = df['score']

In [55]:
x

,budget,company,country,director,genre,gross,rating,runtime,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,R,89,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,PG-13,103,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,PG,110,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,R,137,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,PG,90,Joey Cramer,36636,Mark H. Baker,1986
...,...,...,...,...,...,...,...,...,...,...,...,...
6815,0.0,Fox Searchlight Pictures,UK,Mandie Fletcher,Comedy,4750497.0,R,91,Jennifer Saunders,9161,Jennifer Saunders,2016
6816,0.0,Siempre Viva Productions,USA,Paul Duddridge,Drama,28368.0,PG-13,90,Selma Blair,1959,Paige Cameron,2016
6817,3500000.0,Warner Bros. Animation,USA,Sam Liu,Animation,3775000.0,R,76,Kevin Conroy,36333,Brian Azzarello,2016
6818,0.0,Borderline Presents,USA,Nicolas Pesce,Drama,25981.0,R,76,Kika Magalhães,6947,Nicolas Pesce,2016


In [56]:
y

0       8.1
1       7.8
2       6.9
3       8.4
4       6.9
       ... 
6815    5.4
6816    4.9
6817    6.5
6818    6.2
6819    6.7
Name: score, Length: 6820, dtype: float64

# Validation

In [57]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    #stratify=y, -> Causes error
    random_state=1
)

# Prepro

In [58]:
x.head(3)

,budget,company,country,director,genre,gross,rating,runtime,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,R,89,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,PG-13,103,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,PG,110,Tom Cruise,236909,Jim Cash,1986


In [59]:
str_vars = ['company', 'country', 'director', 'genre', 'rating', 'star', 'writer']
num_vars = ['budget', 'gross', 'runtime', 'votes', 'year']

In [60]:
num_preprocessing = pipeline.Pipeline(steps=[
    ('normalization', QuantileTransformer())
])

str_preprocessing = pipeline.Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('str', str_preprocessing, str_vars)
])

prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('normalization',
                                                  QuantileTransformer())]),
                                 ['budget', 'gross', 'runtime', 'votes',
                                  'year']),
                                ('str',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['company', 'country', 'director', 'genre',
                                  'rating', 'star', 'writer'])])

In [61]:
x_train_prepro = prepro.fit_transform(x_train)
x_val_prepro = prepro.transform(x_val)

In [62]:
from sklearn.ensemble import RandomForestRegressor

In [63]:
model = RandomForestRegressor()
model.fit(x_train_prepro, y_train) # 68secs

RandomForestRegressor()

In [64]:
full_model = pipeline.Pipeline([('Preproccessing', prepro), ('Model', model)])

In [65]:
y_pred = model.predict(x_val_prepro)

In [66]:
print("Accuracy:\t", model.score(x_train_prepro, y_train)*100)
#print("Balanced accuracy:\t", metrics.balanced_accuracy_score(y_val, y_pred)*100)

Accuracy:	 93.63574836349467
